# 在CTR点击率预估中，使用GBDT+LR的原理是什么？

- 简单的LR模型是线性模型，学习能力有限，只能学习用户提供的特征，而人工特征提取成本较高，为了解决人工特征提取成本高，的问题提出了GBDT+LR模型。
- GBDT （Gradient Boosting Decision Tree），是由多棵CART回归树组成。每一颗子树都是一颗回归数，每一次迭代生成一颗子树，每一颗子树的学习目标都是上一课子树的伪残差。
- 在GBDT + LR 的模型中，我们利用GBDT子树的特性进行样本的特征组合和提取。例如我们生成了3颗子树，每颗子树的叶子节点都为4个，我们将每个子树的输出的叶子节点置为1，其他叶子节点置为0.例如我们每颗子树的输出都是在第三个叶子节点，最终我们就可以得到[0,0,1,0,0,0,1,0,0,0,1,0]这样的一个向量。作为该样本的新的特征向量。最终输送给LR模型进行预测。

# Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）

- Wide&Deep由Wide部分和Deep部分组成。
   - Wide部分是广义线性模型：$y=w^T[x,\Phi x]+b$，其中 x 和 $\Phi x$ 表示原始特征与叉乘特征.而记忆（memorization）是指通过一系列人工的特征叉乘（cross-product）来构造高效且可解释的非线性特征。但是如果要泛化的话就需要投入大量的人力进行特征工程。
   - Deep部分则是DNN,只需要将原始的特征进行简单的embedding为低维稠密特征，就可以喂给DNN，然后得到大量的高阶特征的特性。所以泛化（generalization）是指只需要少量的特征工程，就可以得到样本中未出现过的特征组合。

# 在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？

#### 一、FNN  
<div align=center> <img src='./FNN.png' /> </div>
<center> FNN结构 </center>
如上图所示，FNN假设输入的原始样本是一个离散的稀疏矩阵（类似one-hot编码），将数据喂给上层的DNN的时候，需要将高纬稀疏特征映射成低维稠密特征，因此通过 FM 预训练得到的每个特征的隐向量，然后将预训练的向量作为初始化参数，这样能够让模型的参数在初始化的时候就处于较优的位置(训练的目的其实就是为了得到最优的模型参数)，能够加快收敛的过程。

#### 二、Wide&Deep 
<div align=center> <img src='./Wide_Deep.png' /> </div>
<center> Wide&Deep结构 </center>
Wide部分是广义线性模型：$y=w^T[x,\Phi x]+b$，其中 x 和 $\Phi x$ 表示原始特征与叉乘特征。符合广义上的的FM定义一阶特征加上二阶特征。
Deep部分其实就是 FNN，只是 deep 部分中的 embedding 层不用 FM 训练得到的隐向量初始化。

#### 三、DeepFm
<div align=center> <img src='./DeepFM.png' /> </div>
<center>DeepFM结构</center>
DeepFM 先将高维稀疏特征embedding到低维稠密特征，然后再将特征分别喂给FM模型和DNN模型。

#### 四、NFM
<div align=center> <img src='./NFM_all.png' /> </div>
<center> NFM结构 </center>
NFM结构如上图所示，整体的预测公式可以表示为
$$
y = \omega_0 + \sum_{i=1}^n\omega_ix_i + f(x)
$$
其中第一项和第二项是线性回归部分，也就是结构图中的LR模型的部分，第三项$f(x)$则是代表Deep部分，也就是结构右边的部分。其中的BI层又可以表示为
$$
f_{BI}(v_x) = \sum_{i=1}^n\sum_{j=i+1}^n x_iv_i \cdot x_jv_j
$$
由此可以看出NFM的DNN的输入是类似于FM的二阶项，而DeepFM和Wide&Deep模型不同的是DNN层只是原始特征的embedding，而且这二者的DNN层和FM是并行关系。NFM最终可以表示为
<div align=center> <img src='./NFM_fx.png' /> </div>

# GBDT和随机森林都是基于树的算法，它们有什么区别？

#### GBDT（Gradient Boost Decision Tree 梯度提升决策树）
- Boosting Decision Tree：提升树算法是迭代多棵回归树来共同决策。当采用平方误差损失函数时，每一棵回归树学习的是之前所有树的结论和残差，拟合得到一个当前的残差回归树，残差的意义如公式：残差 = 真实值 - 预测值 。提升树即是整个迭代过程生成的回归树的累加。用更通俗一点的说话的话就是类似于一条流失线生成，每一颗树就是流失线上的每一道工序，每一道工序都是在完成前一道工序没有完成的目标。最终所有工序加起来才算最终的完成。
- 对于Boosting Decision Tree来说损失函数，的每一步优化都没那么容易。针对这个缺点提出了GBDT。就是在原有的算法基础上加上了梯度提升算法（Gradient）：利用最速下降的近似方法，即利用损失函数的负梯度在当前模型的值，作为回归问题中提升树算法的残差的近似值，拟合一个回归树。 对应上面的流水线生产的例子就是，为了简化工序，用最少的工序来完成产品，所以在划分工序的时候就将每次区分都最大的地方抽离成一道工序。


#### RF（Random forest 随机森林）
- 随机森林顾名思义，是用随机的方式建立一个森林，森林里面有很多的决策树组成，随机森林的每一棵决策树之间是没有关联的。随机森林的生成可以按照下面四个步骤。
1. 假如有N个样本，则有放回的随机选择N个样本(每次随机选择一个样本，然后返回继续选择)。这选择好了的N个样本用来训练一个决策树，作为决策树根节点处的样本。
2. 当每个样本有M个属性时，在决策树的每个节点需要分裂时，随机从这M个属性中选取出m个属性，满足条件m << M。然后从这m个属性中采用某种策略（比如说信息增益）来选择1个属性作为该节点的分裂属性。
3. 决策树形成过程中每个节点都要按照步骤2来分裂（很容易理解，如果下一次该节点选出来的那一个属性是刚刚其父节点分裂时用过的属性，则该节点已经达到了叶子节点，无须继续分裂了）。一直到不能够再分裂为止。注意整个决策树形成过程中没有进行剪枝
4. 按照步骤1~3建立大量的决策树，这样就构成了随机森林了。
- 每个决策树会得到一个自己的分类结果，决策树的分类结果中哪一个分类最多，那么随机森林就会把这个结果当做最终的结果。通俗的来说，随机森林类似人大代表，只是将选举变为随机，最终在人大代表会议上还是会出现各个行业的代表（也就是第二个步中有各个属性的树），最终结果也是通过投票表决（少数服从多数）。


#### 区别
1. 最主要的区别其实就是集成算法中Boosting（GBDT）和Bagging（RF）的区别，虽然二者都是有放回的抽样，但二者的区别在于：Bagging采用有放回的均匀取样，而Boosting根据错误率来取样（Boosting初始化时对每一个训练样例赋相等的权重1／n，然后用该算法对训练集训练t轮，每次训练后，对训练失败的样例赋以较大的权重），因此Boosting的分类精度要优于Bagging。Bagging的训练集的选择是随机的，各训练集之间相互独立，弱分类器可并行，而Boosting的训练集的选择与前一轮的学习结果有关，是串行的。
2. 组成随机森林的树可以是分类树，也可以是回归树；而GBDT只能由回归树组成。
3. 随机森林的每颗子树都是独立的可以并行生成（就比如我们可以在不同的城市同时选取人大代表）。但是GBDT的子树只能并行生成（流水线生成的时候只能等上一道工序完成才能进行下一道工序）。
4. 随机森林对异常值不敏感（一个人的异常投票并不会影响多数人的决定）；GBDT对异常值非常敏感（一道工序出错了，后面都是在出错的工序上进行在加工）。
5. 对于最终的输出结果而言，随机森林采用多数投票等；而GBDT则是将所有结果累加起来，或者加权累加起来。
6. 随机森林对训练集一视同仁；GBDT是基于权值的弱分类器的集成。

# item流行度在推荐系统中有怎样的应用

1. 冷启动的时候，我们可以基于流行度的推荐
2. 流行度较高的item，较少体现用户个性，流行度较低的item，更能代表用户个性（兴趣），计算用户相似度更准确。在计算相似度推荐的时候，可以推荐相似度高，但不流行的item。